<a href="https://colab.research.google.com/github/puneethkumarvydya/Capstone-Project/blob/Puneeth's-Branch/Capstone_Project_AIML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Importing the required libraries 

import numpy as np 
import pandas as pd